## Assesment question

__Q: Given a jupyter notebook that has a functioning implementation of a machine learning model that identifies unique individuals out of a crowd through gait analysis, how would you translate that notebook to a piece of software that can be used to apply the model to any arbitrary images or videos provided.__

__A:__ As we are dealing with images or videos data, we might probably use neural networks to identify unique individuals. In this case, I would recommend using transfer learning to apply the model to new data. Here are the steps of applying transfer learning:  
1. Train the model on the original task.
2. Change the format of new input data so that they can be passed to the original model.
3. Freeze the layers of the original model but change the output layer as per our task. E.g. the original model has 10 classification outputs while the new task has 2 outputs. We need to change the 10 outputs to 2.
4. Train and fine tune the new model on new data set. We still need to create train and valid set from the new data set.  

The advantage of transfer learning is that we could train the new model on a relatively small data set but get good performance.  
    
For traditional tabular data, we could use joblib or pickle to save the parameters of the model and reload the model to make prediction on new data.  
For both two methods, we need to make sure that the original task and the new task are similar so that the knowledge learned by original model can be transferred to a new context.

## Toy implementation of using CNN to classify hand written digits

In [3]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torchvision import datasets, transforms, utils
from torchsummary import summary
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 14, 'axes.labelweight': 'bold', 'axes.grid': False})

In [4]:
BATCH_SIZE = 256

# Download data
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST('data/', download=True, train=True, transform=transform)
validset = datasets.MNIST('data/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE, shuffle=True)

0it [00:00, ?it/s]

Extracting data/MNIST\raw\train-images-idx3-ubyte.gz to data/MNIST\raw


0it [00:00, ?it/s]

Extracting data/MNIST\raw\train-labels-idx1-ubyte.gz to data/MNIST\raw


0it [00:00, ?it/s]

Extracting data/MNIST\raw\t10k-images-idx3-ubyte.gz to data/MNIST\raw


0it [00:00, ?it/s]

Extracting data/MNIST\raw\t10k-labels-idx1-ubyte.gz to data/MNIST\raw
Processing...
Done!


C:\Users\Maoli\miniconda3\envs\dsci572env\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# Build the CNN architecture

class MNIST_classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 16, (5, 5)),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout(0.2),
            nn.Flatten(),
            nn.Linear(2304, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        out = self.main(x)
        return out

In [6]:
# Create the training function

def trainer(model, criterion, optimizer, trainloader, validloader, epochs=5, verbose=True):
    
    train_loss, valid_loss, valid_accuracy = [], [], []
    for epoch in range(epochs):  
        train_batch_loss = 0
        valid_batch_loss = 0
        valid_batch_acc = 0
        
        # Training
        for X, y in trainloader:
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_batch_loss += loss.item()
        train_loss.append(train_batch_loss / len(trainloader))
        
        # Validation
        model.eval()
        with torch.no_grad(): 
            for X, y in validloader:
                y_hat = model(X)
                _, y_hat_labels = torch.softmax(y_hat, dim=1).topk(1, dim=1)
                loss = criterion(y_hat, y)
                valid_batch_loss += loss.item()
                valid_batch_acc += (y_hat_labels.squeeze() == y).type(torch.float32).mean().item()
        valid_loss.append(valid_batch_loss / len(validloader))
        valid_accuracy.append(valid_batch_acc / len(validloader))  # accuracy
        
        model.train()
        
        # Print progress
        if verbose:
            print(f"Epoch {epoch + 1}:",
                  f"Train Loss: {train_loss[-1]:.3f}.",
                  f"Valid Loss: {valid_loss[-1]:.3f}.",
                  f"Valid Accuracy: {valid_accuracy[-1]:.2f}.")
    
    results = {"train_loss": train_loss,
               "valid_loss": valid_loss,
               "valid_accuracy": valid_accuracy}
    return results

In [7]:
# train the model

model = MNIST_classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
results = trainer(model, criterion, optimizer, trainloader, validloader)

Epoch 1: Train Loss: 0.372. Valid Loss: 0.130. Valid Accuracy: 0.96.
Epoch 2: Train Loss: 0.115. Valid Loss: 0.074. Valid Accuracy: 0.98.
Epoch 3: Train Loss: 0.074. Valid Loss: 0.054. Valid Accuracy: 0.98.
Epoch 4: Train Loss: 0.060. Valid Loss: 0.044. Valid Accuracy: 0.99.
Epoch 5: Train Loss: 0.048. Valid Loss: 0.043. Valid Accuracy: 0.99.
